# Телеграм-бот CV


In [ ]:
!pip install pyTelegramBotAPI

In [2]:
import cv2
import telebot
from google.colab.patches import cv2_imshow
import numpy as np
import PIL


In [ ]:
from data import TOKEN
bot = telebot.TeleBot(TOKEN)

In [ ]:
labels = ["Kate Winslet", "Brad Pitt", "Megan Fox", "Hugh Jackman", "Denzel Washington", "Angelina Jolie", "Jennifer Lawrence", "Natalie Portman", "Johnny Depp",
           "Leonardo DiCaprio", "Robert Downey Jr", "Will Smith",  "Sandra Bullock", "Scarlett Johansson",  "Tom Hanks", "Nicole Kidman"] 
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("/content/drive/MyDrive/Colab Notebooks/ITATTv2/face-art.yml")

font = cv2.FONT_HERSHEY_SIMPLEX 
id = -1

bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Загрузите актера, я попробую распознать лицо')

@bot.message_handler(content_types=['photo'])
def photo_handler(message):
  file = bot.get_file(message.photo[-1].file_id)
  bot.send_message(message.from_user.id, 'Фото получил. Обрабатываю!')
  dfile = bot.download_file(file.file_path)
  fimg = message.photo[-1].file_id + '.png'
  with open(fimg, 'wb') as new_file:
     new_file.write(dfile)
  new_file.close()
  
  img = cv2.imread(fimg)
  gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=2)
  for (x, y, w, h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    gray2 = gray[y:y+h, x:x+w] 
    id, conf = recognizer.predict(gray2) 
    if conf<100:
      id = labels[id]
      conf = "  {0}%".format(round(100 - conf))
      cv2.putText(img, str(id), (x + 5,y - 5), font, 0.6, (0,0,255), 2)
      cv2.putText(img, str(conf), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)
    else:
      idd = "The person is not in the database"
      conf = "  {0}%".format(round(100 - conf))
      cv2.putText(img, str(idd), (x + 5,y - 5), font, 1, (0,0,255), 2)
      cv2.putText(img, str(conf), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)
  cv2_imshow(img) 

  cv2.imwrite(fimg, img)
  with open(fimg,'rb') as fil:
        byte = fil.read()
  print('отправлено')
  bot.send_photo(message.from_user.id,  byte)
  bot.send_message(message.from_user.id, 'Вот кое что нашел!')

bot.infinity_polling()
